# Play 2: Detecting Subtle Sentiment Drift

This notebook demonstrates a more subtle and dangerous scenario: **semantic drift**.

We will compare a baseline dataset of product reviews against a new batch where the star ratings and review lengths appear normal, but the underlying **sentiment** has degraded. This is a classic "silent failure" that traditional monitoring would miss.

### Step 1: Import Libraries and Load Data

In [1]:
import pandas as pd
import whylogs as why
from whylogs.viz import NotebookProfileVisualizer

In [2]:
# Load the two datasets generated by `generate_data.py`
baseline_df = pd.read_csv('data/dataset_baseline.csv')
drifted_df = pd.read_csv('data/dataset_drifted_sentiment.csv')

print("--- Baseline Data Stats ---")
display(baseline_df[['rating', 'review_length', 'sentiment_score']].describe())

print("\n--- Drifted Data Stats ---")
display(drifted_df[['rating', 'review_length', 'sentiment_score']].describe())

--- Baseline Data Stats ---


,rating,review_length,sentiment_score
count,200.000000,200.000000,200.000000
mean,3.875000,159.530000,0.748952
std,1.012175,38.729562,0.202228
min,1.000000,52.000000,0.122000
25%,3.000000,134.000000,0.623925
50%,4.000000,157.500000,0.780200
75%,5.000000,187.500000,0.927525
max,5.000000,262.000000,1.000000



--- Drifted Data Stats ---


,rating,review_length,sentiment_score
count,200.000000,200.000000,200.000000
mean,3.735000,158.840000,0.133879
std,1.113993,41.062086,0.409883
min,1.000000,28.000000,-1.000000
25%,3.000000,135.000000,-0.149050
50%,4.000000,162.000000,0.110400
75%,5.000000,184.250000,0.390350
max,5.000000,260.000000,1.000000


Notice how the statistics for `rating` and `review_length` are nearly identical between the two dataframes. However, the `sentiment_score` distribution has clearly changed. Let's see if `whylogs` can detect this automatically.

### Step 2: Generate Statistical Profiles

In [3]:
# Profile the baseline dataset (this is our reference)
baseline_profile = why.log(baseline_df).profile()

# Profile the new, drifted dataset (this is what we're monitoring)
drifted_profile = why.log(drifted_df).profile()

### Step 3: Visualize the Drift

Now we compare the two profiles. The drift report should flag `sentiment_score` as drifted while showing that `rating` and `review_length` are stable.

In [4]:
viz = NotebookProfileVisualizer()
viz.set_profiles(target_profile_view=drifted_profile.view(), reference_profile_view=baseline_profile.view())

viz.summary_drift_report()

**Success!** The drift report correctly identifies that only the `sentiment_score` feature has drifted, while the other numerical features remain stable. This provides a precise, automated alert for a complex semantic data quality issue.